CTSB - Single gene analysis in GP2 Neurobooster genotyping data (all ancestries)

Project: GP2 CTSB

Version: Python/3.10.15, R/4.3.3

Notebook Overview

Description Loading Python libraries Set paths Make working directory

Installing packages

Create a covariate file with GP2 data

Annotation of the gene CTSB

Association analysis to compare allele frequencies between cases and controls

GLM analysis adjusting for gender, age, PC1-5

Burden test(SkatO, Skat, cmc,zeggini,mb,fp,cmcWald)

Conditional analysis

Loading Python libraries

In [1]:
# Use pathlib for file path manipulation
import pathlib

# Install numpy
import numpy as np

# Install Pandas for tabular data
import pandas as pd

# Install plotnine: a ggplot2-compatible Python plotting package
from plotnine import *

# Always show all columns in a Pandas DataFrame
pd.set_option('display.max_columns', None)

Set paths

In [2]:
REL10_PATH = pathlib.Path(pathlib.Path.home(), 'workspace/gp2_tier2_eu_release10')
!ls -hal {REL10_PATH}

total 100K
dr-xr-xr-x. 1 jupyter users    0 Jul 26 10:56 clinical_data
dr-xr-xr-x. 1 jupyter users    0 Jul 26 10:56 imputed_genotypes
dr-xr-xr-x. 1 jupyter users    0 Jul 26 10:56 meta_data
dr-xr-xr-x. 1 jupyter users    0 Jul 26 10:56 raw_genotypes
dr-xr-xr-x. 1 jupyter users    0 Jul 26 10:56 raw_genotypes_flipped
-r--r--r--. 1 jupyter users 100K Jun 30 20:07 README_release10_01072025.txt
dr-xr-xr-x. 1 jupyter users    0 Jul 26 10:56 wgs


Make working directory

In [3]:
! mkdir ~/workspace/ws_files/CTSB

In [4]:
WORK_DIR = "~/workspace/ws_files/CTSB/"

In [5]:
# make sure all tools installed
! ls /home/jupyter/tools

annovar				       plink2
annovar.latest.tar.gz		       plink2_linux_x86_64_latest.zip
gcta-1.95.0-linux-kernel-3-x86_64      plink_linux_x86_64_20190304.zip
gcta-1.95.0-linux-kernel-3-x86_64.zip  prettify
intel-simplified-software-license.txt  rvtests
LICENSE				       toy.map
__MACOSX			       toy.ped
plink				       vcf_subset


In [6]:
# give permission

# chmod to make sure you have permission to run the program
! chmod u+x /home/jupyter/tools/plink
! chmod u+x /home/jupyter/tools/plink2
! chmod 777 /home/jupyter/tools/rvtests/executable/rvtest

In [7]:
%%bash
# making working directory
#Loop over all the ancestries
for ancestry in {'AAC','AFR','AJ','AMR','CAS','EAS','EUR','FIN','MDE','SAS','CAH'} ;
do

#Make a folder for each ancestry
mkdir ~/workspace/ws_files/CTSB/CTSB_"$ancestry"

done

In [8]:
# covariate file has been created in another notebook without GBA carriers

Annotation of the gene

Extract the region using PLINK

Extract CTSB gene

CTSB coordinates: Chromosome 8: 11,842,524-11,869,533(GRCh38/hg38)

In [9]:
## extract region using plink
ancestries = {'AAC','AFR','AJ','AMR','CAS','EAS','EUR','FIN','MDE','SAS','CAH'}

for ancestry in ancestries:
    
    WORK_DIR = f'~/workspace/ws_files/CTSB/CTSB_{ancestry}'

    ! /home/jupyter/tools/plink2 \
    --pfile {REL10_PATH}/imputed_genotypes/{ancestry}/chr8_{ancestry}_release10_vwb \
    --chr 8 \
    --from-bp 11792524 \
    --to-bp 11919533 \
    --make-bed \
    --out {WORK_DIR}/{ancestry}_CTSB

PLINK v2.0.0-a.7LM 64-bit Intel (7 Jul 2025)       cog-genomics.org/plink/2.0/
(C) 2005-2025 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/workspace/ws_files/CTSB/CTSB_FIN/FIN_CTSB.log.
Options in effect:
  --chr 8
  --from-bp 11792524
  --make-bed
  --out /home/jupyter/workspace/ws_files/CTSB/CTSB_FIN/FIN_CTSB
  --pfile /home/jupyter/workspace/gp2_tier2_eu_release10/imputed_genotypes/FIN/chr8_FIN_release10_vwb
  --to-bp 11919533

Start time: Sat Jul 26 12:49:32 2025
26046 MiB RAM detected, ~24671 available; reserving 13023 MiB for main
workspace.
Using up to 4 compute threads.
144 samples (82 females, 62 males; 144 founders) loaded from
/home/jupyter/workspace/gp2_tier2_eu_release10/imputed_genotypes/FIN/chr8_FIN_release10_vwb.psam.
715411 variants loaded from
/home/jupyter/workspace/gp2_tier2_eu_release10/imputed_genotypes/FIN/chr8_FIN_release10_vwb.pvar.
1 binary phenotype loaded (106 cases, 12 controls).
1057 variants remaining after main

In [10]:
for ancestry in ancestries:
    
    WORK_DIR = f'~/workspace/ws_files/CTSB/CTSB_{ancestry}/'
    
    ! head -n 1 {WORK_DIR}/{ancestry}_CTSB.fam > {WORK_DIR}/{ancestry}_s1.txt

In [ ]:
! head /home/jupyter/workspace/ws_files/CTSB/CTSB_EUR/EUR_s1.txt

Turn binary files into VCF

In [13]:
for ancestry in ancestries:
    
    WORK_DIR = f'~/workspace/ws_files/CTSB/CTSB_{ancestry}'
    
    ## Turn binary files into VCF
    ! /home/jupyter/tools/plink2 \
    --bfile {WORK_DIR}/{ancestry}_CTSB \
    --keep {WORK_DIR}/{ancestry}_s1.txt \
    --make-bed \
    --out {WORK_DIR}/{ancestry}_CTSB_v1

PLINK v2.0.0-a.7LM 64-bit Intel (7 Jul 2025)       cog-genomics.org/plink/2.0/
(C) 2005-2025 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/workspace/ws_files/CTSB/CTSB_FIN/FIN_CTSB_v1.log.
Options in effect:
  --bfile /home/jupyter/workspace/ws_files/CTSB/CTSB_FIN/FIN_CTSB
  --keep /home/jupyter/workspace/ws_files/CTSB/CTSB_FIN/FIN_s1.txt
  --make-bed
  --out /home/jupyter/workspace/ws_files/CTSB/CTSB_FIN/FIN_CTSB_v1

Start time: Sat Jul 26 12:51:59 2025
26046 MiB RAM detected, ~24625 available; reserving 13023 MiB for main
workspace.
Using up to 4 compute threads.
144 samples (82 females, 62 males; 144 founders) loaded from
/home/jupyter/workspace/ws_files/CTSB/CTSB_FIN/FIN_CTSB.fam.
1057 variants loaded from
/home/jupyter/workspace/ws_files/CTSB/CTSB_FIN/FIN_CTSB.bim.
1 binary phenotype loaded (106 cases, 12 controls).
--keep: 1 sample remaining.
1 sample (0 females, 1 male; 1 founder) remaining after main filters.
1 case and 0 controls rem

In [14]:
for ancestry in ancestries:
    
    WORK_DIR = f'~/workspace/ws_files/CTSB/CTSB_{ancestry}'
    
    ## Turn binary files into VCF
    ! /home/jupyter/tools/plink2 \
    --bfile {WORK_DIR}/{ancestry}_CTSB_v1 \
    --recode vcf-fid \
    --out {WORK_DIR}/{ancestry}_CTSB_v1

PLINK v2.0.0-a.7LM 64-bit Intel (7 Jul 2025)       cog-genomics.org/plink/2.0/
(C) 2005-2025 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/workspace/ws_files/CTSB/CTSB_FIN/FIN_CTSB_v1.log.
Options in effect:
  --bfile /home/jupyter/workspace/ws_files/CTSB/CTSB_FIN/FIN_CTSB_v1
  --export vcf-fid
  --out /home/jupyter/workspace/ws_files/CTSB/CTSB_FIN/FIN_CTSB_v1

Start time: Sat Jul 26 12:52:42 2025
Note: --export 'vcf-fid' modifier is deprecated.  Use 'vcf' + 'id-paste=fid'.
26046 MiB RAM detected, ~24629 available; reserving 13023 MiB for main
workspace.
Using up to 4 compute threads.
1 sample (0 females, 1 male; 1 founder) loaded from
/home/jupyter/workspace/ws_files/CTSB/CTSB_FIN/FIN_CTSB_v1.fam.
1057 variants loaded from
/home/jupyter/workspace/ws_files/CTSB/CTSB_FIN/FIN_CTSB_v1.bim.
1 binary phenotype loaded (1 case, 0 controls).
--export vcf to /home/jupyter/workspace/ws_files/CTSB/CTSB_FIN/FIN_CTSB_v1.vcf
.done.
End time: Sat Jul 26 12:

In [15]:
### Bgzip and Tabix (zip and index the file)
for ancestry in ancestries:
    
    WORK_DIR = f'~/workspace/ws_files/CTSB/CTSB_{ancestry}'
    ! bgzip -f {WORK_DIR}/{ancestry}_CTSB_v1.vcf
    ! tabix -f -p vcf {WORK_DIR}/{ancestry}_CTSB_v1.vcf.gz

Annotate using ANNOVAR

In [16]:
## annotate using ANNOVAR

for ancestry in ancestries:
    
    WORK_DIR = f'~/workspace/ws_files/CTSB/CTSB_{ancestry}'
    
    ! perl /home/jupyter/tools/annovar/table_annovar.pl {WORK_DIR}/{ancestry}_CTSB_v1.vcf.gz /home/jupyter/tools/annovar/humandb/ -buildver hg38 \
    -out {WORK_DIR}/{ancestry}_CTSB.annovar \
    -remove -protocol refGene,clinvar_20140902 \
    -operation g,f \
    --nopolish \
    -nastring . \
    -vcfinput


NOTICE: Running with system command <convert2annovar.pl  -includeinfo -allsample -withfreq -format vcf4 /home/jupyter/workspace/ws_files/CTSB/CTSB_FIN/FIN_CTSB_v1.vcf.gz > /home/jupyter/workspace/ws_files/CTSB/CTSB_FIN/FIN_CTSB.annovar.avinput>
NOTICE: Finished reading 1064 lines from VCF file
NOTICE: A total of 1057 locus in VCF file passed QC threshold, representing 1008 SNPs (625 transitions and 383 transversions) and 49 indels/substitutions
NOTICE: Finished writing allele frequencies based on 1008 SNP genotypes (625 transitions and 383 transversions) and 49 indels/substitutions for 1 samples

NOTICE: Running with system command </home/jupyter/tools/annovar/table_annovar.pl /home/jupyter/workspace/ws_files/CTSB/CTSB_FIN/FIN_CTSB.annovar.avinput /home/jupyter/tools/annovar/humandb/ -buildver hg38 -outfile /home/jupyter/workspace/ws_files/CTSB/CTSB_FIN/FIN_CTSB.annovar -remove -protocol refGene,clinvar_20140902 -operation g,f --nopolish -nastring . -otherinfo>
-----------------------

In [17]:
WORK_DIR = f'~/workspace/ws_files/CTSB/'

In [48]:
# Read in ANNOVAR multianno file
gene = pd.read_csv(f'{WORK_DIR}/CTSB_SAS/SAS_CTSB.annovar.hg38_multianno.txt', sep = '\t')
display(gene)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,clinvar_20140902,Otherinfo1,Otherinfo2,Otherinfo3,Otherinfo4,Otherinfo5,Otherinfo6,Otherinfo7,Otherinfo8,Otherinfo9,Otherinfo10,Otherinfo11,Otherinfo12,Otherinfo13
0,8,11792575,11792575,T,C,intergenic,"NEIL2,FDFT1",dist=5230;dist=2998,.,.,.,0,.,.,8,11792575,chr8:11792575:T:C,T,C,.,.,PR,GT,0/0
1,8,11792593,11792593,T,G,intergenic,"NEIL2,FDFT1",dist=5248;dist=2980,.,.,.,0,.,.,8,11792593,chr8:11792593:T:G,T,G,.,.,PR,GT,0/0
2,8,11792765,11792765,T,A,intergenic,"NEIL2,FDFT1",dist=5420;dist=2808,.,.,.,0,.,.,8,11792765,chr8:11792765:T:A,T,A,.,.,PR,GT,0/0
3,8,11792784,11792784,G,A,intergenic,"NEIL2,FDFT1",dist=5439;dist=2789,.,.,.,0.5,.,.,8,11792784,chr8:11792784:G:A,G,A,.,.,PR,GT,0/1
4,8,11792794,11792794,T,C,intergenic,"NEIL2,FDFT1",dist=5449;dist=2779,.,.,.,0,.,.,8,11792794,chr8:11792794:T:C,T,C,.,.,PR,GT,0/0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2796,8,11919376,11919376,A,T,intergenic,"CTSB,DEFB136",dist=51289;dist=54561,.,.,.,0,.,.,8,11919376,chr8:11919376:A:T,A,T,.,.,PR,GT,0/0
2797,8,11919450,11919450,G,C,intergenic,"CTSB,DEFB136",dist=51363;dist=54487,.,.,.,0,.,.,8,11919450,chr8:11919450:G:C,G,C,.,.,PR,GT,0/0
2798,8,11919469,11919469,T,C,intergenic,"CTSB,DEFB136",dist=51382;dist=54468,.,.,.,0,.,.,8,11919469,chr8:11919469:T:C,T,C,.,.,PR,GT,0/0
2799,8,11919475,11919475,T,C,intergenic,"CTSB,DEFB136",dist=51388;dist=54462,.,.,.,0,.,.,8,11919475,chr8:11919475:T:C,T,C,.,.,PR,GT,0/0


In [49]:
gene["Func.refGene"].value_counts()

Func.refGene
intronic      1436
intergenic    1002
UTR5           109
UTR3            97
upstream        58
exonic          56
downstream      43
Name: count, dtype: int64

In [50]:
# Filter exonic variants
coding = gene[gene['Func.refGene'] == 'exonic']
coding["ExonicFunc.refGene"].value_counts()

ExonicFunc.refGene
nonsynonymous SNV         38
synonymous SNV            17
nonframeshift deletion     1
Name: count, dtype: int64

In [51]:
#Make lists of variants to keep - all coding, coding nonsynonymous (missense - as they are coded in ANNOVAR), deleterious (CADD_phred > 20)

for ancestry in ancestries:
    
    WORK_DIR = f'~/workspace/ws_files/CTSB/CTSB_{ancestry}'
    print(f'WORKING ON: {ancestry}')

    # Read in ANNOVAR multianno file
    gene = pd.read_csv(f'{WORK_DIR}/{ancestry}_CTSB.annovar.hg38_multianno.txt', sep = '\t')
    
    #Print number of variants in the different categories
    results = [] 

    utr5 = gene[gene['Func.refGene']== 'UTR5']
    intronic = gene[gene['Func.refGene']== 'intronic']
    exonic = gene[gene['Func.refGene']== 'exonic']
    utr3 = gene[gene['Func.refGene']== 'UTR3']
    coding_nonsynonymous = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'nonsynonymous SNV')]
    coding_synonymous = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] != 'nonsynonymous SNV')]
    lof = exonic[(exonic['ExonicFunc.refGene'] == 'stopgain') | (exonic['ExonicFunc.refGene'] == 'stoploss') | (exonic['ExonicFunc.refGene'] == 'frameshift deletion') | (exonic['ExonicFunc.refGene'] == 'frameshift insertion')]
    nonsynonymous_lof = pd.concat([coding_nonsynonymous, lof])

    print({ancestry})
    print('Total variants: ', len(gene))
    print("Intronic: ", len(intronic))
    print('UTR3: ', len(utr3))
    print('UTR5: ', len(utr5))
    print("Total exonic: ", len(exonic))
    print('  Synonymous: ', len(coding_synonymous))
    print("  Nonsynonymous: ", len(coding_nonsynonymous))
    print("nonsynonymous_lof: ", len(nonsynonymous_lof))
    results.append((gene, intronic, utr3, utr5, exonic, coding_synonymous, coding_nonsynonymous, nonsynonymous_lof))
    print('\n')

    # Save in PLINK format - coding nonsynonymous 
    # These are missense variants - other types of nonsynonymous variants (e.g stopgain/loss, or frameshift variants are coded differently in the ExonicFunc.refGene 
    variants_toKeep = nonsynonymous_lof[['Chr', 'Start', 'End', 'Gene.refGene']].copy()
    variants_toKeep.to_csv(f'{WORK_DIR}/{ancestry}_CTSB.nonsynonymous_lof.variantstoKeep.txt', sep="\t", index=False, header=False)


    # Save in PLINK format - all coding variants
    variants_toKeep2 = exonic[['Chr', 'Start', 'End', 'Gene.refGene']].copy()
    variants_toKeep2.to_csv(f'{WORK_DIR}/{ancestry}_CTSB.exonic.variantstoKeep.txt', sep="\t", index=False, header=False)

WORKING ON: FIN
{'FIN'}
Total variants:  1057
Intronic:  556
UTR3:  37
UTR5:  43
Total exonic:  16
  Synonymous:  7
  Nonsynonymous:  9
nonsynonymous_lof:  9


WORKING ON: AMR
{'AMR'}
Total variants:  5507
Intronic:  2903
UTR3:  184
UTR5:  224
Total exonic:  90
  Synonymous:  38
  Nonsynonymous:  52
nonsynonymous_lof:  53


WORKING ON: MDE
{'MDE'}
Total variants:  3639
Intronic:  1928
UTR3:  107
UTR5:  129
Total exonic:  58
  Synonymous:  22
  Nonsynonymous:  36
nonsynonymous_lof:  36


WORKING ON: AAC
{'AAC'}
Total variants:  5526
Intronic:  2972
UTR3:  182
UTR5:  197
Total exonic:  75
  Synonymous:  36
  Nonsynonymous:  39
nonsynonymous_lof:  39


WORKING ON: EUR
{'EUR'}
Total variants:  10388
Intronic:  5374
UTR3:  352
UTR5:  415
Total exonic:  211
  Synonymous:  78
  Nonsynonymous:  133
nonsynonymous_lof:  137


WORKING ON: CAH
{'CAH'}
Total variants:  5508
Intronic:  2957
UTR3:  177
UTR5:  203
Total exonic:  83
  Synonymous:  39
  Nonsynonymous:  44
nonsynonymous_lof:  45


WORKIN

ALL variants

assoc

glm

ASSOC

In [52]:
#Run case-control analysis using plink assoc for all variants, not adjusting for any covariates
ancestries = ['AAC', 'AFR', 'AJ', 'AMR', 'CAS', 'EAS', 'EUR', 'FIN', 'MDE', 'SAS', 'CAH']

for ancestry in ancestries:

    WORK_DIR = f'~/workspace/ws_files/CTSB/CTSB_{ancestry}'
    
    ! /home/jupyter/tools/plink \
    --bfile {WORK_DIR}/{ancestry}_CTSB \
    --keep /home/jupyter/workspace/ws_files/{ancestry}/{ancestry}.noGBA.samplestoKeep \
    --assoc \
    --allow-no-sex \
    --ci 0.95 \
    --maf 0.01 \
    --out {WORK_DIR}/{ancestry}_CTSB.all

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/workspace/ws_files/CTSB/CTSB_AAC/AAC_CTSB.all.log.
Options in effect:
  --allow-no-sex
  --assoc
  --bfile /home/jupyter/workspace/ws_files/CTSB/CTSB_AAC/AAC_CTSB
  --ci 0.95
  --keep /home/jupyter/workspace/ws_files/AAC/AAC.noGBA.samplestoKeep
  --maf 0.01
  --out /home/jupyter/workspace/ws_files/CTSB/CTSB_AAC/AAC_CTSB.all

26046 MB RAM detected; reserving 13023 MB for main workspace.
5526 variants loaded from .bim file.
1215 people (532 males, 683 females) loaded from .fam.
1133 phenotype values loaded from .fam.
--keep: 1207 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1207 founders and 0 nonfounders present.
Calculating allele frequencies... done.
Total genotyping rate in remaining samples is 0.997149.
4315 variants removed due to minor allele t

In [53]:
ancestries = ['AAC', 'AFR', 'AJ', 'AMR', 'CAS', 'EAS', 'EUR', 'FIN', 'MDE', 'SAS', 'CAH']

for ancestry in ancestries:
    
    WORK_DIR = f'~/workspace/ws_files/CTSB/CTSB_{ancestry}'
    print(f'WORKING ON: {ancestry}')
    
    #Look at assoc results, only variants with nominal p-value < 0.05
    freq = pd.read_csv(f'{WORK_DIR}/{ancestry}_CTSB.all.assoc', sep='\s+')
    sig_all_nonadj = freq[freq['P']<0.05]
    
    print(f'Variants with p-value < 0.05: {sig_all_nonadj.shape}')
    
    #Save FREQ to csv
    freq.to_csv(f'{WORK_DIR}/{ancestry}.all_nonadj.csv')

WORKING ON: AAC
Variants with p-value < 0.05: (112, 13)
WORKING ON: AFR
Variants with p-value < 0.05: (81, 13)
WORKING ON: AJ
Variants with p-value < 0.05: (23, 13)
WORKING ON: AMR
Variants with p-value < 0.05: (433, 13)
WORKING ON: CAS
Variants with p-value < 0.05: (86, 13)
WORKING ON: EAS
Variants with p-value < 0.05: (129, 13)
WORKING ON: EUR
Variants with p-value < 0.05: (272, 13)
WORKING ON: FIN
Variants with p-value < 0.05: (6, 13)
WORKING ON: MDE
Variants with p-value < 0.05: (46, 13)
WORKING ON: SAS
Variants with p-value < 0.05: (7, 13)
WORKING ON: CAH
Variants with p-value < 0.05: (31, 13)


In [54]:
#Run case-control analysis with covariates
ancestries = ['AAC', 'AFR', 'AJ', 'AMR', 'CAS', 'EAS', 'EUR', 'FIN', 'MDE', 'SAS', 'CAH']

for ancestry in ancestries:

    WORK_DIR = f'~/workspace/ws_files/CTSB/CTSB_{ancestry}'

    ! /home/jupyter/tools/plink2 \
    --bfile {WORK_DIR}/{ancestry}_CTSB \
    --keep /home/jupyter/workspace/ws_files/{ancestry}/{ancestry}.noGBA.samplestoKeep \
    --allow-no-sex \
    --maf 0.01 \
    --ci 0.95 \
    --glm \
    --covar /home/jupyter/workspace/ws_files/{ancestry}/{ancestry}.noGBA.noNA.txt \
    --covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5 \
    --covar-variance-standardize \
    --neg9-pheno-really-missing \
    --out {WORK_DIR}/{ancestry}_CTSB.all_adj

PLINK v2.0.0-a.7LM 64-bit Intel (7 Jul 2025)       cog-genomics.org/plink/2.0/
(C) 2005-2025 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/workspace/ws_files/CTSB/CTSB_AAC/AAC_CTSB.all_adj.log.
Options in effect:
  --allow-no-sex
  --bfile /home/jupyter/workspace/ws_files/CTSB/CTSB_AAC/AAC_CTSB
  --ci 0.95
  --covar /home/jupyter/workspace/ws_files/AAC/AAC.noGBA.noNA.txt
  --covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5
  --covar-variance-standardize
  --glm
  --keep /home/jupyter/workspace/ws_files/AAC/AAC.noGBA.samplestoKeep
  --maf 0.01
  --neg9-pheno-really-missing
  --out /home/jupyter/workspace/ws_files/CTSB/CTSB_AAC/AAC_CTSB.all_adj

Start time: Sat Jul 26 13:03:18 2025
Note: --allow-no-sex no longer has any effect.  (Missing-sex samples are
automatically excluded from association analysis when sex is a covariate, and
treated normally otherwise.)
26046 MiB RAM detected, ~24620 available; reserving 13023 MiB for main
workspace.
Using up to 4 c

In [55]:
WORK_DIR = f'~/workspace/ws_files/CTSB/CTSB_AMR'

! /home/jupyter/tools/plink2 \
--bfile {WORK_DIR}/AMR_CTSB \
--keep /home/jupyter/workspace/ws_files/AMR/AMR.noGBA.samplestoKeep \
--allow-no-sex \
--maf 0.01 \
--ci 0.95 \
--glm \
--covar /home/jupyter/workspace/ws_files/AMR/AMR_covariate_file_noGBA.orthoPCs.txt \
--covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5 \
--covar-variance-standardize \
--neg9-pheno-really-missing \
--out {WORK_DIR}/AMR_CTSB.all_adj

PLINK v2.0.0-a.7LM 64-bit Intel (7 Jul 2025)       cog-genomics.org/plink/2.0/
(C) 2005-2025 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/workspace/ws_files/CTSB/CTSB_AMR/AMR_CTSB.all_adj.log.
Options in effect:
  --allow-no-sex
  --bfile /home/jupyter/workspace/ws_files/CTSB/CTSB_AMR/AMR_CTSB
  --ci 0.95
  --covar /home/jupyter/workspace/ws_files/AMR/AMR_covariate_file_noGBA.orthoPCs.txt
  --covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5
  --covar-variance-standardize
  --glm
  --keep /home/jupyter/workspace/ws_files/AMR/AMR.noGBA.samplestoKeep
  --maf 0.01
  --neg9-pheno-really-missing
  --out /home/jupyter/workspace/ws_files/CTSB/CTSB_AMR/AMR_CTSB.all_adj

Start time: Sat Jul 26 13:04:36 2025
Note: --allow-no-sex no longer has any effect.  (Missing-sex samples are
automatically excluded from association analysis when sex is a covariate, and
treated normally otherwise.)
26046 MiB RAM detected, ~24638 available; reserving 13023 MiB for main
workspa

In [56]:
#Process results from plink glm analysis including covariates
ancestries = ['AAC', 'AFR', 'AJ', 'AMR', 'CAS', 'EAS', 'EUR', 'FIN', 'MDE', 'SAS', 'CAH']

for ancestry in ancestries:

    WORK_DIR = f'~/workspace/ws_files/CTSB/CTSB_{ancestry}'
    print(f'WORKING ON: {ancestry}')
    
    #Read in plink glm results
    assoc = pd.read_csv(f'{WORK_DIR}/{ancestry}_CTSB.all_adj.PHENO1.glm.logistic.hybrid', delim_whitespace=True)

    #Filter for additive test only - this is the variant results
    assoc_add = assoc[assoc['TEST']=="ADD"]
    
    #Check if there are any significant (p < 0.05) variants
    significant = assoc_add[assoc_add['P']<0.05]

    print(f'There are {len(significant)} variants with p-value < 0.05 in glm')
    
    #Check if there are any significant (p < 0.05) variants
    GWsignificant = assoc_add[assoc_add['P']<5e-8]

    print(f'There are {len(GWsignificant)} variants with p-value < 5e-8 in glm')
    
    #Save assoc_add to csv
    assoc_add.to_csv(f'{WORK_DIR}/{ancestry}.all_adj.csv')

WORKING ON: AAC
There are 51 variants with p-value < 0.05 in glm
There are 0 variants with p-value < 5e-8 in glm


/tmp/ipykernel_1691/829170199.py:10: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead


WORKING ON: AFR
There are 94 variants with p-value < 0.05 in glm
There are 0 variants with p-value < 5e-8 in glm


/tmp/ipykernel_1691/829170199.py:10: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead


WORKING ON: AJ
There are 34 variants with p-value < 0.05 in glm
There are 0 variants with p-value < 5e-8 in glm
WORKING ON: AMR


...

/tmp/ipykernel_1691/829170199.py:10: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
/tmp/ipykernel_1691/829170199.py:10: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead


There are 0 variants with p-value < 0.05 in glm
There are 0 variants with p-value < 5e-8 in glm
WORKING ON: MDE
There are 49 variants with p-value < 0.05 in glm
There are 0 variants with p-value < 5e-8 in glm
WORKING ON: SAS
There are 3 variants with p-value < 0.05 in glm
There are 0 variants with p-value < 5e-8 in glm


/tmp/ipykernel_1691/829170199.py:10: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead


WORKING ON: CAH
There are 33 variants with p-value < 0.05 in glm
There are 0 variants with p-value < 5e-8 in glm


/tmp/ipykernel_1691/829170199.py:10: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead


format assoc file to do meta analysis(AFR/AJ/AMR/EAS/EUR)

In [57]:
# Define ancestry groups and their sample sizes
ancestry_sample_sizes = {
    'AFR': 3316,
    'AJ': 1693,
    'AMR': 3319,
    'EAS': 4975,
    'EUR': 33870
}

# Output container (optional)
cleaned_results = {}

# Loop over each ancestry
for ancestry, sample_size in ancestry_sample_sizes.items():
    # Load PLINK association result file
    file_path = f'/home/jupyter/workspace/ws_files/CTSB/CTSB_{ancestry}/{ancestry}.all_adj.csv'
    df = pd.read_csv(file_path)

    # Build MARKERNAME
    df['MARKERNAME'] = 'chr' + df['#CHROM'].astype(str) + ':' + df['POS'].astype(str)

    # Rename columns
    df.rename(columns={
        '#CHROM': 'CHROMOSOME',
        'POS': 'POSITION',
        'REF': 'NEA',
        'ALT': 'EA',
        'A1_FREQ': 'EAF',
        'LOG(OR)_SE': 'SE',
        'L95': 'OR_95L',
        'U95': 'OR_95U'
    }, inplace=True)

    # Add BETA = log(OR)
    df['BETA'] = np.log(df['OR'])

    # Add sample size
    df['N'] = sample_size

    # Keep desired columns
    final_cols = [
        'MARKERNAME', 'CHROMOSOME', 'POSITION', 'NEA', 'EA', 'EAF',
        'BETA', 'OR', 'SE', 'OR_95L', 'OR_95U', 'P', 'N'
    ]
    df_cleaned = df[final_cols]

    # Store or save
    cleaned_results[ancestry] = df_cleaned

    # Optional: save to file
    output_path = f'/home/jupyter/workspace/ws_files/CTSB/CTSB_{ancestry}/{ancestry}.cleaned_assoc.txt.gz'
    df_cleaned.to_csv(output_path, sep='\t', index=False)

In [58]:
df = pd.read_csv('/home/jupyter/workspace/ws_files/CTSB/CTSB_AJ/AJ.cleaned_assoc.txt.gz', sep='\t')
df

,MARKERNAME,CHROMOSOME,POSITION,NEA,EA,EAF,BETA,OR,SE,OR_95L,OR_95U,P,N
0,chr8:11792575,8,11792575,T,C,0.019353,0.071269,1.073870,0.370647,0.519347,2.220490,0.847515,1693
1,chr8:11792812,8,11792812,C,T,0.114708,-0.096086,0.908386,0.159408,0.664633,1.241540,0.546664,1693
2,chr8:11792966,8,11792966,T,C,0.398739,-0.033624,0.966935,0.107804,0.782771,1.194430,0.755114,1693
3,chr8:11793099,8,11793099,T,C,0.154628,0.170721,1.186160,0.154521,0.876222,1.605730,0.269231,1693
4,chr8:11793200,8,11793200,G,C,0.129552,0.097363,1.102260,0.159246,0.806741,1.506030,0.540933,1693
...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,chr8:11918618,8,11918618,T,C,0.064459,-0.002321,0.997682,0.218229,0.650482,1.530200,0.991517,1693
738,chr8:11918643,8,11918643,T,G,0.015526,-0.228613,0.795636,0.410404,0.355941,1.778490,0.577496,1693
739,chr8:11919062,8,11919062,A,G,0.019649,-0.845029,0.429545,0.306669,0.235490,0.783512,0.005860,1693
740,chr8:11919200,8,11919200,G,A,0.083690,-0.260764,0.770463,0.177479,0.544103,1.090990,0.141761,1693


Burden Analyses using RVTests

In [59]:
ancestries = ['AAC', 'AFR', 'AJ', 'AMR', 'CAS', 'EAS', 'EUR', 'FIN', 'MDE', 'SAS', 'CAH']
variant_classes = ['exonic', 'nonsynonymous_lof']

#Loop over all the ancestries and the 2 variant classes - run rvtests for all coding and missense variants
for ancestry in ancestries:
    for variant_class in variant_classes:
        
        WORK_DIR = f'~/workspace/ws_files/CTSB/CTSB_{ancestry}'

        # Print the command to be executed (for debugging purposes)
        print(f'Running plink to extract {variant_class} variants for ancestry: {ancestry}')
        
        #Extract relevant variants
        ! /home/jupyter/tools/plink2 \
        --pfile {REL10_PATH}/imputed_genotypes/{ancestry}/chr8_{ancestry}_release10_vwb \
        --keep /home/jupyter/workspace/ws_files/{ancestry}/{ancestry}.noGBA.samplestoKeep \
        --extract range {WORK_DIR}/{ancestry}_CTSB.{variant_class}.variantstoKeep.txt \
        --recode vcf-iid \
        --out {WORK_DIR}/{ancestry}_CTSB.{variant_class}
        
        # Print the command to be executed (for debugging purposes)
        print(f'Running bgzip and tabix for {variant_class} variants for ancestry: {ancestry}')
        
        ## Bgzip and Tabix (zip and index the file)
        ! bgzip -f {WORK_DIR}/{ancestry}_CTSB.{variant_class}.vcf
        ! tabix -f -p vcf {WORK_DIR}/{ancestry}_CTSB.{variant_class}.vcf.gz

Running plink to extract exonic variants for ancestry: AAC
PLINK v2.0.0-a.7LM 64-bit Intel (7 Jul 2025)       cog-genomics.org/plink/2.0/
(C) 2005-2025 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/workspace/ws_files/CTSB/CTSB_AAC/AAC_CTSB.exonic.log.
Options in effect:
  --export vcf-iid
  --extract range /home/jupyter/workspace/ws_files/CTSB/CTSB_AAC/AAC_CTSB.exonic.variantstoKeep.txt
  --keep /home/jupyter/workspace/ws_files/AAC/AAC.noGBA.samplestoKeep
  --out /home/jupyter/workspace/ws_files/CTSB/CTSB_AAC/AAC_CTSB.exonic
  --pfile /home/jupyter/workspace/gp2_tier2_eu_release10/imputed_genotypes/AAC/chr8_AAC_release10_vwb

Start time: Sat Jul 26 13:08:44 2025
Note: --export 'vcf-iid' modifier is deprecated.  Use 'vcf' + 'id-paste=iid'.
26046 MiB RAM detected, ~24623 available; reserving 13023 MiB for main
workspace.
Using up to 4 compute threads.
1215 samples (683 females, 532 males; 1215 founders) loaded from
/home/jupyter/workspace/gp2_t

In [ ]:
#Run RVtests
ancestries = ['AAC', 'AFR', 'AJ', 'AMR', 'CAS', 'EAS', 'EUR', 'FIN', 'MDE', 'SAS', 'CAH']
variant_classes = ['exonic', 'nonsynonymous_lof']

for ancestry in ancestries:
    for variant_class in variant_classes:
        
        WORK_DIR = f'~/workspace/ws_files/CTSB/CTSB_{ancestry}'

        # Print the command to be executed (for debugging purposes)
        print(f'Running RVtests for {variant_class} variants for ancestry: {ancestry}')
        
        ## RVtests with covariates 
        #Make sure the pheno and covariate file starts with the first 5 columsn: fid, iid, fatid, matid, sex
        #The pheno-name flag only works when the pheno/covar file is structured properly
        ! /home/jupyter/tools/rvtests/executable/rvtest --noweb --hide-covar \
        --out {WORK_DIR}/{ancestry}_CTSB.burden.{variant_class} \
        --kernel skato \
        --inVcf {WORK_DIR}/{ancestry}_CTSB.{variant_class}.vcf.gz \
        --pheno /home/jupyter/workspace/ws_files/{ancestry}/{ancestry}_covariate_file_noGBA.txt \
        --pheno-name PHENO \
        --gene CTSB \
        --geneFile /home/jupyter/workspace/ws_files/refFlat_HG38.txt \
        --covar /home/jupyter/workspace/ws_files/{ancestry}/{ancestry}_covariate_file_noGBA.txt \
        --covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5 \
        --freqUpper 0.01
# --burden cmc,zeggini,mb,fp,cmcWald --kernel skat,skato \

Look at RVtest results SKAT-O

In [61]:
%load_ext rpy2.ipython

/opt/conda/lib/python3.10/site-packages/rpy2/rinterface/__init__.py:1211: UserWarning: Environment variable "R_SESSION_TMPDIR" redefined by R and overriding existing variable. Current: "/tmp/RtmpZB9niY", R: "/tmp/RtmpWw6ESi"


In [62]:
WORK_DIR = "~/workspace/ws_files/CTSB"

In [63]:
#Check EUR all_coding variant results
! cat {WORK_DIR}/CTSB_EUR/EUR_CTSB.burden.exonic.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
CTSB	8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868087,8:11842524-11868150,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479971-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568	40128	93	81	230895	0	0.57025


In [64]:
#Check EUR nonsynonymous_lof variant results
! cat {WORK_DIR}/CTSB_EUR/EUR_CTSB.burden.nonsynonymous_lof.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
CTSB	8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868087,8:11842524-11868150,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479971-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568	40128	59	55	145610	1	0.558984


In [67]:
#Check AAC all_coding variant results
! cat {WORK_DIR}/CTSB_AAC/AAC_CTSB.burden.exonic.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
CTSB	8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868087,8:11842524-11868150,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479971-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568	1061	28	26	17177.6	0	0.747416


In [68]:
#Check AAC nonsynonymous_lof variant results
! cat {WORK_DIR}/CTSB_AAC/AAC_CTSB.burden.nonsynonymous_lof.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
CTSB	8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868087,8:11842524-11868150,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479971-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568	1061	15	15	21269.3	0	0.27747


In [71]:
#Check AFR all_coding variant results
! cat {WORK_DIR}/CTSB_AFR/AFR_CTSB.burden.exonic.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
CTSB	8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868087,8:11842524-11868150,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479971-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568	2738	46	29	15284.2	0	0.83222


In [72]:
#Check AFR nonsynonymous_lof variant results
! cat {WORK_DIR}/CTSB_AFR/AFR_CTSB.burden.nonsynonymous_lof.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
CTSB	8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868087,8:11842524-11868150,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479971-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568	2738	25	18	44255	0.3	0.145056


In [73]:
# First, define your p-values in Python
pvalues = [0.83222, 0.145056]
# Send to R
import rpy2.robjects as ro
ro.globalenv['pvalues'] = ro.FloatVector(pvalues)

In [74]:
%%R
# Adjust using BH method
fdrs <- p.adjust(pvalues, method = "BH")
print(fdrs)

[1] 0.832220 0.290112


In [75]:
#Check AJ all_coding variant results
! cat {WORK_DIR}/CTSB_AJ/AJ_CTSB.burden.exonic.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
CTSB	8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868087,8:11842524-11868150,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479971-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568	2174	13	11	4309.19	1	0.728951


In [76]:
#Check AJ nonsynonymous_lof variant results
! cat {WORK_DIR}/CTSB_AJ/AJ_CTSB.burden.nonsynonymous_lof.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
CTSB	8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868087,8:11842524-11868150,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479971-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568	2174	11	9	1176.1	1	0.881028


In [77]:
# First, define your p-values in Python
pvalues = [0.728951, 0.881028]
# Send to R
import rpy2.robjects as ro
ro.globalenv['pvalues'] = ro.FloatVector(pvalues)

In [78]:
%%R
# Adjust using BH method
fdrs <- p.adjust(pvalues, method = "BH")
print(fdrs)

[1] 0.881028 0.881028


In [79]:
#Check AMR all_coding variant results
! cat {WORK_DIR}/CTSB_AMR/AMR_CTSB.burden.exonic.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
CTSB	8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868087,8:11842524-11868150,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479971-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568	3265	39	26	17400.8	0	0.745868


In [80]:
#Check AMR nonsynonymous_lof variant results
! cat {WORK_DIR}/CTSB_AMR/AMR_CTSB.burden.nonsynonymous_lof.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
CTSB	8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868087,8:11842524-11868150,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479971-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568	3265	27	19	17122.4	0	0.471545


In [81]:
# First, define your p-values in Python
pvalues = [0.745868, 0.471545]
# Send to R
import rpy2.robjects as ro
ro.globalenv['pvalues'] = ro.FloatVector(pvalues)

In [82]:
%%R
# Adjust using BH method
fdrs <- p.adjust(pvalues, method = "BH")
print(fdrs)

[1] 0.745868 0.745868


In [83]:
#Check CAS all_coding variant results
! cat {WORK_DIR}/CTSB_CAS/CAS_CTSB.burden.exonic.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
CTSB	8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868087,8:11842524-11868150,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479971-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568	1104	18	11	814.855	1	0.857092


In [84]:
#Check CAS nonsynonymous_lof variant results
! cat {WORK_DIR}/CTSB_CAS/CAS_CTSB.burden.nonsynonymous_lof.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
CTSB	8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868087,8:11842524-11868150,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479971-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568	1104	12	9	1996.05	1	0.756806


In [85]:
# First, define your p-values in Python
pvalues = [0.857092, 0.756806]
# Send to R
import rpy2.robjects as ro
ro.globalenv['pvalues'] = ro.FloatVector(pvalues)

In [86]:
%%R
# Adjust using BH method
fdrs <- p.adjust(pvalues, method = "BH")
print(fdrs)

[1] 0.857092 0.857092


In [87]:
#Check EAS all_coding variant results
! cat {WORK_DIR}/CTSB_EAS/EAS_CTSB.burden.exonic.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
CTSB	8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868087,8:11842524-11868150,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479971-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568	4670	24	14	704.747	1	1


In [88]:
#Check EAS nonsynonymous_lof variant results
! cat {WORK_DIR}/CTSB_EAS/EAS_CTSB.burden.nonsynonymous_lof.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
CTSB	8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868087,8:11842524-11868150,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479971-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568	4670	17	10	11348.2	1	0.539251


In [89]:
# First, define your p-values in Python
pvalues = [1, 0.539251]
# Send to R
import rpy2.robjects as ro
ro.globalenv['pvalues'] = ro.FloatVector(pvalues)

In [90]:
%%R
# Adjust using BH method
fdrs <- p.adjust(pvalues, method = "BH")
print(fdrs)

[1] 1 1


In [91]:
#Check FIN all_coding variant results
! cat {WORK_DIR}/CTSB_FIN/FIN_CTSB.burden.exonic.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
CTSB	8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868087,8:11842524-11868150,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479971-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568	45	3	0	NA	NA	NA


In [92]:
#Check FIN nonsynonymous_lof variant results
! cat {WORK_DIR}/CTSB_FIN/FIN_CTSB.burden.nonsynonymous_lof.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
CTSB	8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868087,8:11842524-11868150,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479971-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568	45	2	0	NA	NA	NA


In [93]:
#Check MDE all_coding variant results
! cat {WORK_DIR}/CTSB_MDE/MDE_CTSB.burden.exonic.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
CTSB	8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868087,8:11842524-11868150,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479971-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568	1230	24	14	11656.6	1	0.400443


In [94]:
#Check MDE nonsynonymous_lof variant results
! cat {WORK_DIR}/CTSB_MDE/MDE_CTSB.burden.nonsynonymous_lof.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
CTSB	8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868087,8:11842524-11868150,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479971-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568	1230	15	7	2204.48	1	0.565363


In [95]:
# First, define your p-values in Python
pvalues = [0.400443, 0.565363]
# Send to R
import rpy2.robjects as ro
ro.globalenv['pvalues'] = ro.FloatVector(pvalues)

In [96]:
%%R
# Adjust using BH method
fdrs <- p.adjust(pvalues, method = "BH")
print(fdrs)

[1] 0.565363 0.565363


In [97]:
#Check SAS all_coding variant results
! cat {WORK_DIR}/CTSB_SAS/SAS_CTSB.burden.exonic.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
CTSB	8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868087,8:11842524-11868150,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479971-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568	689	19	8	2239.37	1	0.419244


In [98]:
#Check SAS nonsynonymous_lof variant results
! cat {WORK_DIR}/CTSB_SAS/SAS_CTSB.burden.nonsynonymous_lof.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
CTSB	8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868087,8:11842524-11868150,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479971-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568	689	12	5	774.291	0	0.639828


In [99]:
# First, define your p-values in Python
pvalues = [0.419244, 0.639828]
# Send to R
import rpy2.robjects as ro
ro.globalenv['pvalues'] = ro.FloatVector(pvalues)

In [100]:
%%R
# Adjust using BH method
fdrs <- p.adjust(pvalues, method = "BH")
print(fdrs)

[1] 0.639828 0.639828


In [101]:
#Check CAH all_coding variant results
! cat {WORK_DIR}/CTSB_CAH/CAH_CTSB.burden.exonic.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
CTSB	8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868087,8:11842524-11868150,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479971-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568	900	28	21	18155.3	0	0.730282


In [102]:
#Check CAH nonsynonymous_lof variant results
! cat {WORK_DIR}/CTSB_CAH/CAH_CTSB.burden.nonsynonymous_lof.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
CTSB	8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868150,8:11842524-11868087,8:11842524-11868150,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479971-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568,8_KZ208915v1_fix:1479908-1505568	900	14	12	3469.22	1	0.817762


In [103]:
# First, define your p-values in Python
pvalues = [0.730282, 0.817762]
# Send to R
import rpy2.robjects as ro
ro.globalenv['pvalues'] = ro.FloatVector(pvalues)

In [104]:
%%R
# Adjust using BH method
fdrs <- p.adjust(pvalues, method = "BH")
print(fdrs)

[1] 0.817762 0.817762


LD pruning(in EUR)

In [105]:
WORK_DIR = "~/workspace/ws_files/"

In [106]:
# Make sure to use high-quality SNPs
! /home/jupyter/tools/plink2 \
--bfile {WORK_DIR}/CTSB/CTSB_EUR/EUR_CTSB \
--maf 0.01 \
--geno 0.05 \
--hwe 1e-5 0.001 \
--make-bed \
--keep /home/jupyter/workspace/ws_files/EUR/EUR.noGBA.samplestoKeep \
--exclude {WORK_DIR}/exclusion_regions_hg38.txt \
--out {WORK_DIR}/CTSB/CTSB_UNIMPUTED

PLINK v2.0.0-a.7LM 64-bit Intel (7 Jul 2025)       cog-genomics.org/plink/2.0/
(C) 2005-2025 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/workspace/ws_files//CTSB/CTSB_UNIMPUTED.log.
Options in effect:
  --bfile /home/jupyter/workspace/ws_files//CTSB/CTSB_EUR/EUR_CTSB
  --exclude /home/jupyter/workspace/ws_files//exclusion_regions_hg38.txt
  --geno 0.05
  --hwe 1e-5 0.001
  --keep /home/jupyter/workspace/ws_files/EUR/EUR.noGBA.samplestoKeep
  --maf 0.01
  --make-bed
  --out /home/jupyter/workspace/ws_files//CTSB/CTSB_UNIMPUTED

Start time: Sat Jul 26 13:26:22 2025
26046 MiB RAM detected, ~24630 available; reserving 13023 MiB for main
workspace.
Using up to 4 compute threads.
58823 samples (25749 females, 33074 males; 58823 founders) loaded from
/home/jupyter/workspace/ws_files//CTSB/CTSB_EUR/EUR_CTSB.fam.
10388 variants loaded from
/home/jupyter/workspace/ws_files//CTSB/CTSB_EUR/EUR_CTSB.bim.
1 binary phenotype loaded (26778 cases, 10372 con

In [107]:
# Prune out unnecessary SNPs (only need to do this to generate PCs)
! /home/jupyter/tools/plink2 \
--bfile {WORK_DIR}/CTSB/CTSB_UNIMPUTED \
--indep-pairwise 50 5 0.5 \
--out {WORK_DIR}/CTSB/prune_CTSB

PLINK v2.0.0-a.7LM 64-bit Intel (7 Jul 2025)       cog-genomics.org/plink/2.0/
(C) 2005-2025 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/workspace/ws_files//CTSB/prune_CTSB.log.
Options in effect:
  --bfile /home/jupyter/workspace/ws_files//CTSB/CTSB_UNIMPUTED
  --indep-pairwise 50 5 0.5
  --out /home/jupyter/workspace/ws_files//CTSB/prune_CTSB

Start time: Sat Jul 26 13:26:47 2025
26046 MiB RAM detected, ~24571 available; reserving 13023 MiB for main
workspace.
Using up to 4 compute threads.
53641 samples (23385 females, 30256 males; 53641 founders) loaded from
/home/jupyter/workspace/ws_files//CTSB/CTSB_UNIMPUTED.fam.
599 variants loaded from
/home/jupyter/workspace/ws_files//CTSB/CTSB_UNIMPUTED.bim.
1 binary phenotype loaded (24208 cases, 9662 controls).
Calculating allele frequencies... done.
483/599 variants removed.ead): 0%
Variant lists written to
/home/jupyter/workspace/ws_files//CTSB/prune_CTSB.prune.in and
/home/jupyter/workspace/

In [108]:
!wc -l {WORK_DIR}/CTSB/prune_CTSB.prune.in

116 /home/jupyter/workspace/ws_files//CTSB/prune_CTSB.prune.in


In [109]:
# 4.31E-4

zoom locus plot

In [2]:
## locus zoom format
ancestries = {'AAC','AFR','AJ','AMR','CAS','EAS','EUR','FIN','MDE','SAS','CAH'}

for ancestry in ancestries:
    WORK_DIR = f'~/workspace/ws_files/CTSB/CTSB_{ancestry}'

    df = pd.read_csv(f'{WORK_DIR}/{ancestry}.all_adj.csv')
    df['b'] = np.log(df['OR'])
    export_ldassoc = df[['#CHROM', 'POS', 'A1', 'OMITTED', 'A1_FREQ', 'b', 'LOG(OR)_SE', 'P']].copy()
    export_ldassoc = export_ldassoc.rename(columns = {'#CHROM': 'CHROM', 'OMITTED':'A2', 'A1_FREQ':'freq', 'LOG(OR)_SE':'se', 'P':'p'})
    # save to files
    export_ldassoc.to_csv(f'{WORK_DIR}/{ancestry}.all_adj.formatted.tab', sep = '\t', index=False)

GCTA cojo analysis

In [111]:
## cojo format
ancestries = {'AAC','AFR','AJ','AMR','CAS','EAS','EUR','FIN','MDE','SAS','CAH'}

for ancestry in ancestries:
    WORK_DIR = f'~/workspace/ws_files/CTSB/CTSB_{ancestry}'

    sumstats = pd.read_csv(f'{WORK_DIR}/{ancestry}.all_adj.csv')
    #Format summary statistics for GCTA-COJO
    #First get the log odds ratio - this is required for COJO
    #1) For a case-control study, the effect size should be log(odds ratio) with its corresponding standard error.
    sumstats_formatted = sumstats.copy()
    sumstats_formatted['b'] = np.log(sumstats_formatted['OR'])

    #Now select just the necessary columns for COJO
    sumstats_export = sumstats_formatted[['ID', 'A1', 'OMITTED', 'A1_FREQ', 'b', 'LOG(OR)_SE', 'P', 'OBS_CT']].copy()

    #Rename columns following COJO format
    sumstats_export = sumstats_export.rename(columns = {'ID':'SNP', 'OMITTED':'A2', 'A1_FREQ':'freq', 'LOG(OR)_SE':'se', 'P':'p', 'OBS_CT':'N'})

    #Export
    sumstats_export.to_csv(f'{WORK_DIR}/{ancestry}.all_adj.sumstats.ma', sep = '\t', index=False)

In [112]:
ancestries = {'AAC','AFR','AJ','AMR','CAS','EAS','EUR','FIN','MDE','SAS','CAH'}

for ancestry in ancestries:
    
    WORK_DIR = f'~/workspace/ws_files/CTSB/CTSB_{ancestry}'
    
    #Select multiple associated SNPs based on LD pruning p-value 4.67e-4
    #Can change the p-value for significance if needed
    #bfile is referring to the full dataset in plink binary format, e.g. GP2 - whatever you used to run the GWAS
    ! /home/jupyter/tools/gcta-1.95.0-linux-kernel-3-x86_64/gcta64 \
    --bfile {WORK_DIR}/{ancestry}_CTSB \
    --maf 0.01 \
    --cojo-file {WORK_DIR}/{ancestry}.all_adj.sumstats.ma \
    --cojo-p 4.31e-4 \
    --cojo-slct \
    --out {WORK_DIR}/{ancestry}.all_adj.ldprune.COJO

*******************************************************************
* Genome-wide Complex Trait Analysis (GCTA)
* version v1.95.0 Linux
* Built at Jul 21 2025 17:30:18, by GCC 8.4
* (C) 2010-present, Yang Lab, Westlake University
* Please report bugs to Jian Yang <jian.yang@westlake.edu.cn>
*******************************************************************
Analysis started at 13:29:35 UTC on Sat Jul 26 2025.
Hostname: 4b8bf4fabeb0

Accepted options:
--bfile /home/jupyter/workspace/ws_files/CTSB/CTSB_FIN/FIN_CTSB
--maf 0.01
--cojo-file /home/jupyter/workspace/ws_files/CTSB/CTSB_FIN/FIN.all_adj.sumstats.ma
--cojo-p 0.000431
--cojo-slct
--out /home/jupyter/workspace/ws_files/CTSB/CTSB_FIN/FIN.all_adj.ldprune.COJO


Reading PLINK FAM file from [/home/jupyter/workspace/ws_files/CTSB/CTSB_FIN/FIN_CTSB.fam].
144 individuals to be included from [/home/jupyter/workspace/ws_files/CTSB/CTSB_FIN/FIN_CTSB.fam].
Reading PLINK BIM file from [/home/jupyter/workspace/ws_files/CTSB/CTSB_FIN/FIN_CTSB.bi